## In this notebook, we're going to cover some of the most fundamental concepts of tensors using Tensorflow

More specifically, we're going to cover:
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function (a way to speed up your regular Python function)
* Using GPUs with TensorFlow (or TPUs)
* Exercises to try

### Introduction to TensorFlow

In [ ]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__)

2.11.0


In [ ]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [ ]:
# Check the number of dimensions of a tensor (ndim stans for number of dimensions)
scalar.ndim

0

In [ ]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [ ]:
# Check the dimensions of the vectors
vector.ndim

1

In [ ]:
# Create a matrix
matrix = tf.constant([[10, 7],
                      [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [ ]:
# Check the dimensions of the matrix
matrix.ndim

2

In [ ]:
# Create another matrix to play around woth the `dtype` parameter. Normally, it is int32
another_matrix = tf.constant([[10., 7.0],
                              [3.0, 2.0],
                              [8.0, 9.0]], dtype = tf.float16) #specify the data type with the dtype parameter
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [ ]:
# Check the dimensions for another_matrix
another_matrix.ndim

2

In [ ]:
# Let's create a tensor
tensor = tf.constant([[[1, 2, 3],
                       [4, 5, 6]],
                      [[7, 8, 9],
                       [10, 11, 12]],
                      [[13, 14, 15],
                       [16, 17, 18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [ ]:
tensor.ndim

3

What we've created so far:

* Scalar: a single number
* Vector: a number with direction (eg. wind speed and direction)
* Matrix: a 2-dimensional array of numbers
* Tensor: an n-dimensional array of numbers (where n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector)

### Creating tensors with `tf.Variable`

In [ ]:
# Creating the same tensor with tf.Variable() as above
changable_tensor = tf.Variable([10, 7])
unchangable_tensor = tf.constant([10, 7])
changable_tensor, unchangable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [ ]:
# # Let's try change one of the elements in our changable tensor
# changable_tensor[0] = 7
# changable_tensor
# Doesnot work 

In [ ]:
# How about we try .assign()
changable_tensor[0].assign(7)
changable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [ ]:
# # Let's try the same on the unchangable tensor 
# unchangable_tensor[0].assign(7)
# unchangable_tensor
# Doesnot work 

**Note:** Rarely in practise will you need to decide wether to use `tf.constant()` or `tf.Variable()` to create tensorts, as TensorFlow does this for you. However, if in doubt, use `tf.constant()` and change later if needed.

### Creating random tensors 

Random tensors are tensors of some arbitary size which contain random numbers 

In [ ]:
# Create two random (but same) tensors 
random_1 = tf.random.Generator.from_seed(42) # set seed for reproducability 
random_1 = random_1.normal(shape= (3, 2))
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape = (3, 2))

# Are they equal?
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

In [ ]:
# Shuffle a tensor (valuable for when you want to shuffle your data so the inherent order doesn't affect learning)
not_shuffled = tf.constant([[10, 7],
                            [3, 4],
                            [2, 5]])

# Shuffle our non-shuffled tensor 
tf.random.shuffle(not_shuffled)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [ ]:
# Shuffle our non-shuffled tensor 
tf.random.set_seed(42)
tf.random.shuffle(not_shuffled, seed = 42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

**Exercise:** Read through TensorFlow documentation on random seed generation: https://www.tensorflow.org/api_docs/python/tf/random/set_seed and practise writing 5 random tensors and shuffle them.

It looks like if we want our shuffled tensors to be in the same order, we've got to use the global level random seed as well as the operation level random seed.

> Rule 4: "If both the global and the operation seed are set: Both seed are used in conjuction to determine the random sequence"

In [ ]:
t1 = tf.constant([1])
t2 = tf.constant([1, 2, 3])
t3  = tf.constant([[1, 2, 3],
                   [4, 5, 6]])
t4 = tf.constant([[[1, 2, 3],
                   [4, 5, 6]],
                  [[7, 8, 9],
                   [10, 11, 12]],
                  [[13, 14, 15],
                   [16, 17, 18]]])
t5 = tf.Variable([5, 9, 3])

In [ ]:
tf.random.set_seed(42)
tf.random.shuffle(t1, seed = 42)

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>

In [ ]:
tf.random.set_seed(42)
tf.random.shuffle(t2, seed = 42)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>

In [ ]:
tf.random.set_seed(77)
tf.random.shuffle(t3, seed = 77)

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]], dtype=int32)>

In [ ]:
tf.random.set_seed(65)
tf.random.shuffle(t4, seed = 65)

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]],

       [[ 1,  2,  3],
        [ 4,  5,  6]]], dtype=int32)>

In [ ]:
tf.random.set_seed(42)
tf.random.shuffle(t5, seed = 42)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([5, 9, 3], dtype=int32)>

### Other ways to make tensors 




In [ ]:
# Create tensor of all ones
tf.ones([10, 7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [ ]:
# Create a tensor of all zeros
tf.zeros(shape = (3, 4))

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### Turn NumPy arrays into tensors

The main difference between NumPy arrays and TensorFlow tensors is that the tensors can be run on a GPU (much faster for numerical computing )

In [ ]:
# You can also turn NumPy arrays into tensors 
import numpy as np
numpy_A = np.arange(1, 25, dtype = np.int32) # Create a NumPy array between 1 and 25
numpy_A

# X = tf.constant(some_matrix) # capital for matrix or tensor
# y = tf.constant(vector) # non-capital for vector

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [ ]:
A = tf.constant(numpy_A, shape = (3, 4, 2))
B = tf.constant(numpy_A)
A, B

(<tf.Tensor: shape=(3, 4, 2), dtype=int32, numpy=
 array([[[ 1,  2],
         [ 3,  4],
         [ 5,  6],
         [ 7,  8]],
 
        [[ 9, 10],
         [11, 12],
         [13, 14],
         [15, 16]],
 
        [[17, 18],
         [19, 20],
         [21, 22],
         [23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

### Getting information from tensors 


When dealing with tensors you probably want to be aware of the following attributes:
* Shape
* Rank
* Axis or dimension
* Size

In [ ]:
# Create a rank 4 tensor (4-dimensions)
rank_4_tensor = tf.zeros(shape= (2, 3, 4, 5))

In [ ]:
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [ ]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [ ]:
# Get various attributes of our tensor
print("Datatype of every element: ", rank_4_tensor.dtype)
print("Number of dimensions (rank): ", rank_4_tensor.ndim)
print("Shape of tensor: ", rank_4_tensor.shape)
print("Elements along the 0 axis: ", rank_4_tensor.shape[0])
print("Elements along the last axis: ", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor: ", tf.size(rank_4_tensor).numpy())

Datatype of every element:  <dtype: 'float32'>
Number of dimensions (rank):  4
Shape of tensor:  (2, 3, 4, 5)
Elements along the 0 axis:  2
Elements along the last axis:  5
Total number of elements in our tensor:  120


### Indexing tensors

Tensors can be indexed just like Python lists.

In [ ]:
some_list = [1, 2, 3, 4]
some_list[:2]

[1, 2]

In [ ]:
# Get the first 2 elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [ ]:
# Get each dimension from each index except for the final one
rank_4_tensor[:1, :1, :1]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [ ]:
rank_4_tensor[:1, :1, :, :1]

<tf.Tensor: shape=(1, 1, 4, 1), dtype=float32, numpy=
array([[[[0.],
         [0.],
         [0.],
         [0.]]]], dtype=float32)>

In [ ]:
# Create a rank 2 tensor
rank_2_tensor = tf.constant([[10, 7],
                             [3, 4]])
rank_2_tensor.shape, rank_2_tensor.ndim

(TensorShape([2, 2]), 2)

In [ ]:
# Get the last item of each of our rank 2 tensor 
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4], dtype=int32)>

In [ ]:
# Add in extra dimension to our rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [ ]:
# Alternative to tf.newaxis()
tf.expand_dims(rank_2_tensor, axis = -1) # "-1" means expand the final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [ ]:
tf.expand_dims(rank_2_tensor, axis = 0) # expand the 0-axis

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[10,  7],
        [ 3,  4]]], dtype=int32)>

In [ ]:
tf.expand_dims(rank_2_tensor, axis = 1) # expand the 1-axis

<tf.Tensor: shape=(2, 1, 2), dtype=int32, numpy=
array([[[10,  7]],

       [[ 3,  4]]], dtype=int32)>

### Manipulating tensor (tensor operations)

**Basic Operations**

`+`, `-`, `*`, `/`

In [ ]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[10, 7],
                      [3, 4]])
tensor+10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [ ]:
# Original tensor is unchanged
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [ ]:
# Multiplication also works
tensor*10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [ ]:
# Subraction if you want
tensor-10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

In [ ]:
# We can use the tensorflow built-in function too
tf.multiply(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

Now, what's the difference? The difference being the fact that when you use TensorFlow, the code can be executed on a GPU and hence, the speed of execution is more as compared to the normal conventional methods 

In [ ]:
tensor 

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

### **Matrix Multiplication**

In machine learning, matrix multiplication is one of the most common tensor operations

There are rules our tensors (or matrices) need to fulfill if we're going to matrix multiply them:

1. The inner dimensions must match
2. the resulting matrix has the shape of outer dimensions.

In [ ]:
# Matrix multiplication in TensorFlow
print(tensor)
tf.matmul(tensor, tensor)

tf.Tensor(
[[10  7]
 [ 3  4]], shape=(2, 2), dtype=int32)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [ ]:
# Matrix multiplication with Python operator "@"
tensor @ tensor 

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [ ]:
# Create a tensor (3, 2) tensor
X = tf.constant([[1, 2],
                 [3, 4],
                 [5, 6]])

# Create a tensor (3, 2) tensor 
Y = tf.constant([[7, 8],
                 [9, 10],
                 [11, 12]])
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>, <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [ ]:
# # Try to matrix multiply tensors of same shape
# X @ Y
# Does not work 

In [ ]:
# Let's change the shape of Y
tf.reshape(Y, shape = (2, 3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 7,  8,  9],
       [10, 11, 12]], dtype=int32)>

In [ ]:
# Try to matrix multiply X by the reshaped Y
X @ tf.reshape(Y, shape = (2, 3))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [ ]:
tf.matmul(X, tf.reshape(Y, shape = (2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [ ]:
# Try changing the shape of X instead of Y
tf.matmul(tf.reshape(X, shape = (2, 3)), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 58,  64],
       [139, 154]], dtype=int32)>

In [ ]:
# Can you do the same with transpose
X, tf.transpose(X), tf.reshape(X, shape = (2, 3))

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>, <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 5],
        [2, 4, 6]], dtype=int32)>, <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>)

In [ ]:
# Try matrix multiplication with transpose rathen than reshape
tf.matmul(tf.transpose(X), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

**The dot product: **

Matrix multiplication is also referred to as the dot product.

You can perform matrix multiplication using:
* tf.matmul()
* tf.tensordot()

In [ ]:
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>, <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [ ]:
# Perform the dot product on X and Y (requires X or Y to be transposed)
tf.tensordot(tf.transpose(X), Y, axes = 1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

In [ ]:
# Perform matrix multiplication between X and Y (transposed)
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [ ]:
# Perform matrix multiplication between X and Y (reshaped)
tf.matmul(X, tf.reshape(Y, shape= (2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

### Changing the datatype of a tensor 

In [ ]:
tf.__version__

'2.11.0'

In [ ]:
# Create a new tensor with default datatype (float32)
B = tf.constant([1.7, 7.4])
B, B.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.7, 7.4], dtype=float32)>,
 tf.float32)

In [ ]:
C = tf.constant([7, 10])
C.dtype

tf.int32

In [ ]:
# Change from float32 to float 16 (reduced precision)
B = tf.cast(B, dtype = tf.float16)
B, B.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>,
 tf.float16)

In [ ]:
# Change from int32 to float32
E = tf.cast(C, dtype = tf.float32)
E, E.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 7., 10.], dtype=float32)>,
 tf.float32)

In [ ]:
E_float = tf.cast(E, dtype = tf.float16)
E_float

<tf.Tensor: shape=(2,), dtype=float16, numpy=array([ 7., 10.], dtype=float16)>

### Aggregating

Aggregating tensors = condensing them from multiple values down to a smaller amount of values.

In [ ]:
# Get the absolute values
D = tf.constant([-10, -7])
D

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([-10,  -7], dtype=int32)>

In [ ]:
# Get the absolute numbers
tf.abs(D)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>

Let's go through the following forms of aggregation

* Get the minimum
* Get the maximum
* Get the mean of a tensor
* Get the sum of a tensor 

In [ ]:
# Create a random tensor with values between 0 and 100 of size 50
E = tf.constant(np.random.randint(0, 100, size = 50))
E

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([ 0, 97, 18, 29, 35, 37, 73, 70, 11, 80, 13, 97, 85,  2, 91, 99, 44,
       15, 79, 56, 77, 80, 83, 60, 51, 32,  2, 13, 43,  7, 79, 86, 52,  4,
       79, 36, 43, 94, 36, 95,  1, 91,  1, 56,  9, 98, 10, 75, 31, 47])>

In [ ]:
tf.size(E), E.shape, E.ndim

(<tf.Tensor: shape=(), dtype=int32, numpy=50>, TensorShape([50]), 1)

In [ ]:
# Find the minimum
tf.reduce_min(E)

<tf.Tensor: shape=(), dtype=int64, numpy=0>

In [ ]:
# Find the maximum
tf.reduce_max(E)

<tf.Tensor: shape=(), dtype=int64, numpy=99>

In [ ]:
# Find the mean
tf.reduce_mean(E)

<tf.Tensor: shape=(), dtype=int64, numpy=50>

In [ ]:
# Find the sum
tf.reduce_sum(E)

<tf.Tensor: shape=(), dtype=int64, numpy=2502>

In [ ]:
 # Find the variance of our tensor 
import tensorflow_probability as tfp
tfp.stats.variance(E)

<tf.Tensor: shape=(), dtype=int64, numpy=1091>

In [ ]:
# Find the standard deviation
tf.math.reduce_std(tf.cast(E, dtype = tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=33.03269>

In [ ]:
# Find the vartiance of E tensor 
tf.math.reduce_variance(tf.cast(E, dtype = tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=1091.1584>

### Find the positional minimum and maximum 

In [ ]:
# Create a new tensor for finding positional minimum and maximum
tf.random.set_seed(42)
F = tf.random.uniform(shape = [50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [ ]:
# Find the positional maximum i.e. the position at which the maximum value occurs
tf.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [ ]:
# Index on our largest value position
F[tf.argmax(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [ ]:
# Find the max value of F
tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [ ]:
# Check for equality 
assert F[tf.argmax(F)] == tf.reduce_max(F)

In [ ]:
# Find the minimum using the positional minimum index
F[tf.argmin(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.009463668>

### Squezing a tensor (Removing all single dimensions)

In [ ]:
# Create a tensor to get started 
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape = [50]), shape = (1, 1, 1, 1, 50))
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
           0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
           0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
           0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
           0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
           0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
           0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
           0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
           0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
           0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043]]]]],
      dtype=float32)>

In [ ]:
G.shape

TensorShape([1, 1, 1, 1, 50])

In [ ]:
G_squeezed = tf.squeeze(G)
G_squeezed, G_squeezed.shape

(<tf.Tensor: shape=(50,), dtype=float32, numpy=
 array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
        0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
        0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
        0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
        0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
        0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
        0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
        0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
        0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
        0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
       dtype=float32)>, TensorShape([50]))

### One-hot encoding tensors

In [ ]:
# Create a list of indices
some_list = [0, 1, 2, 3]

# One hot encode our list of indices 
tf.one_hot(some_list, depth = 4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [ ]:
tf.one_hot(some_list, depth = 3)

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 0.]], dtype=float32)>

In [ ]:
tf.one_hot(some_list, depth = 6)

<tf.Tensor: shape=(4, 6), dtype=float32, numpy=
array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.]], dtype=float32)>

In [ ]:
# Specify custom values for one hot encoding 
tf.one_hot(some_list, depth = 4, on_value = "AHHHH YEAHHHH", off_value= "OHHHHH NOOOOO")

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'AHHHH YEAHHHH', b'OHHHHH NOOOOO', b'OHHHHH NOOOOO',
        b'OHHHHH NOOOOO'],
       [b'OHHHHH NOOOOO', b'AHHHH YEAHHHH', b'OHHHHH NOOOOO',
        b'OHHHHH NOOOOO'],
       [b'OHHHHH NOOOOO', b'OHHHHH NOOOOO', b'AHHHH YEAHHHH',
        b'OHHHHH NOOOOO'],
       [b'OHHHHH NOOOOO', b'OHHHHH NOOOOO', b'OHHHHH NOOOOO',
        b'AHHHH YEAHHHH']], dtype=object)>

### Squaring, log, root

In [ ]:
# Create a new tensor 
H = tf.range(1, 10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [ ]:
# Square it
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [ ]:
# Find the square root 
tf.sqrt(tf.cast(H, dtype = tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.2360678, 2.4494896,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [ ]:
# Find the log 
tf.math.log(tf.cast(H, dtype = tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

### Tensors and NumPy
TensorFlow interacts beautifully with NumPy arrays

**Note:** One of the main differences between a TensorFlow tensor and a NumPy array is that a TensorFlow tensor can be run on a GPU or TPU (for faster numerical processing).

In [ ]:
# Create a tensor directly from a NumPy array
J = tf.constant(np.array([3.0, 7.0, 4.0]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([3., 7., 4.])>

In [ ]:
# Convert our tensor back to a NumPy array
np.array(J)

array([3., 7., 4.])

In [ ]:
# Convert tensor J to a NumPy array
J.numpy(), type(J.numpy())

(array([3., 7., 4.]), numpy.ndarray)

In [ ]:
J = tf.constant([3.0])
J.numpy()[0]

3.0

In [ ]:
# The default types of each are slightly different 
numpy_J = tf.constant(np.array([3.0, 7.0, 10.0]))
tensor_J = tf.constant([3.0, 7.0, 10.0])

# Check the datatypes of each 
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

### Finding access to GPUs

In [ ]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
# To check if any GPU is available 
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

The above two cell of code were before connecting to a GPU
The next cell of code is after connecting to a GPU

In [ ]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
# To know more about the GPU you are running on:
!nvidia-smi

Wed Feb 15 08:21:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    30W /  70W |    328MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Note:** If you have access to a CUDA-enabled GPU, TensorFlow will automatically use it whenever possible

### TensorFlow Fundamentals Exercises
* Create a vector, scalar, matrix and tensor with values of your choosing using tf.constant().
* Find the shape, rank and size of the tensors you created in 1.
* Create two tensors containing random values between 0 and 1 with shape [5, 300].
* Multiply the two tensors you created in 3 using matrix multiplication.
* Multiply the two tensors you created in 3 using dot product.
* Create a tensor with random values between 0 and 1 with shape [224, 224, 3].
* Find the min and max values of the tensor you created in 6 along the first axis.
* Created a tensor with random values of shape [1, 224, 224, 3] then squeeze it to change the shape to [224, 224, 3].
* Create a tensor with shape [10] using your own choice of values, then find the index which has the maximum value.
* One-hot encode the tensor you created in 9.

In [ ]:
K = tf.constant([7])
L = tf.constant([5, 6])
M = tf.constant([[1, 2],
                 [3, 4]])

In [ ]:
K.shape, L.shape, M.shape

(TensorShape([1]), TensorShape([2]), TensorShape([2, 2]))

In [ ]:
tf.rank(K), tf.rank(L), tf.rank(M)

(<tf.Tensor: shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: shape=(), dtype=int32, numpy=2>)

In [ ]:
tf.size(K), tf.size(L), tf.size(M)

(<tf.Tensor: shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: shape=(), dtype=int32, numpy=2>,
 <tf.Tensor: shape=(), dtype=int32, numpy=4>)

In [ ]:
N = tf.constant(np.random.uniform(0, 1, size= [5, 300]))
O = tf.constant(np.random.uniform(0, 1, size= [5, 300]))

In [ ]:
tf.matmul(N, tf.transpose(O))

<tf.Tensor: shape=(5, 5), dtype=float64, numpy=
array([[78.32420906, 75.52128765, 82.33583781, 76.03673007, 76.53404232],
       [76.71444048, 78.05857998, 79.12095387, 80.16309576, 78.79440157],
       [77.24870053, 77.11811902, 77.30520646, 75.48425504, 76.0628503 ],
       [69.70820167, 71.3705345 , 71.00027609, 72.85833124, 71.04402364],
       [77.25730172, 74.99667312, 77.42348784, 79.06249791, 73.41940789]])>

In [ ]:
N @ tf.transpose(O)

<tf.Tensor: shape=(5, 5), dtype=float64, numpy=
array([[78.32420906, 75.52128765, 82.33583781, 76.03673007, 76.53404232],
       [76.71444048, 78.05857998, 79.12095387, 80.16309576, 78.79440157],
       [77.24870053, 77.11811902, 77.30520646, 75.48425504, 76.0628503 ],
       [69.70820167, 71.3705345 , 71.00027609, 72.85833124, 71.04402364],
       [77.25730172, 74.99667312, 77.42348784, 79.06249791, 73.41940789]])>

In [ ]:
P = tf.constant(np.random.uniform(0, 1, size = [224, 224, 3]))
P

<tf.Tensor: shape=(224, 224, 3), dtype=float64, numpy=
array([[[0.01158129, 0.68775613, 0.85824461],
        [0.2863575 , 0.15274254, 0.26746166],
        [0.20745616, 0.53103351, 0.91808157],
        ...,
        [0.21046013, 0.1252665 , 0.85810826],
        [0.94902596, 0.79512993, 0.03802893],
        [0.04042776, 0.08004032, 0.40284664]],

       [[0.18425903, 0.78555556, 0.87351645],
        [0.13318865, 0.20400351, 0.48521524],
        [0.55408783, 0.07622929, 0.6158169 ],
        ...,
        [0.54811126, 0.34401812, 0.70106532],
        [0.08739242, 0.20344557, 0.93385123],
        [0.7551669 , 0.17638656, 0.59804404]],

       [[0.26784164, 0.45726943, 0.70707221],
        [0.49949095, 0.34327342, 0.46143584],
        [0.91316467, 0.5667217 , 0.38084576],
        ...,
        [0.62653714, 0.84653871, 0.51198849],
        [0.23339963, 0.52617392, 0.08952852],
        [0.68919481, 0.73258792, 0.68817275]],

       ...,

       [[0.63341363, 0.21010257, 0.5863131 ],
        [0.08

In [ ]:
tf.reduce_max(P)

<tf.Tensor: shape=(), dtype=float64, numpy=0.9999994592144994>

In [ ]:
tf.reduce_min(P)

<tf.Tensor: shape=(), dtype=float64, numpy=2.350089064129257e-06>

In [ ]:
Q = tf.constant(np.random.uniform(0, 1, size = [1, 224, 224, 3]))
Q

<tf.Tensor: shape=(1, 224, 224, 3), dtype=float64, numpy=
array([[[[0.75323099, 0.3160619 , 0.50335016],
         [0.25235376, 0.13682958, 0.46447875],
         [0.04081702, 0.73126545, 0.37204808],
         ...,
         [0.57398059, 0.21410258, 0.80477732],
         [0.66213264, 0.09847787, 0.39244393],
         [0.42039118, 0.04804504, 0.98349801]],

        [[0.66682826, 0.76506428, 0.19714795],
         [0.23964496, 0.78206175, 0.79825844],
         [0.45847857, 0.07966685, 0.83482387],
         ...,
         [0.90317151, 0.47681889, 0.27545741],
         [0.35623448, 0.39178472, 0.75446395],
         [0.4952731 , 0.46429311, 0.76519526]],

        [[0.26099447, 0.23574925, 0.72522352],
         [0.65585159, 0.31691214, 0.90783722],
         [0.93054291, 0.13704543, 0.60950067],
         ...,
         [0.37611221, 0.35886459, 0.64296131],
         [0.83709991, 0.65234251, 0.33494395],
         [0.60424715, 0.33062849, 0.77721707]],

        ...,

        [[0.40359709, 0.3585796 , 

In [ ]:
tf.squeeze(Q), tf.size(tf.squeeze(Q))

(<tf.Tensor: shape=(224, 224, 3), dtype=float64, numpy=
 array([[[0.75323099, 0.3160619 , 0.50335016],
         [0.25235376, 0.13682958, 0.46447875],
         [0.04081702, 0.73126545, 0.37204808],
         ...,
         [0.57398059, 0.21410258, 0.80477732],
         [0.66213264, 0.09847787, 0.39244393],
         [0.42039118, 0.04804504, 0.98349801]],
 
        [[0.66682826, 0.76506428, 0.19714795],
         [0.23964496, 0.78206175, 0.79825844],
         [0.45847857, 0.07966685, 0.83482387],
         ...,
         [0.90317151, 0.47681889, 0.27545741],
         [0.35623448, 0.39178472, 0.75446395],
         [0.4952731 , 0.46429311, 0.76519526]],
 
        [[0.26099447, 0.23574925, 0.72522352],
         [0.65585159, 0.31691214, 0.90783722],
         [0.93054291, 0.13704543, 0.60950067],
         ...,
         [0.37611221, 0.35886459, 0.64296131],
         [0.83709991, 0.65234251, 0.33494395],
         [0.60424715, 0.33062849, 0.77721707]],
 
        ...,
 
        [[0.40359709, 0.3585796 

In [ ]:
R = tf.constant([4, 7, 97, 65, 43, 43, 66, 9, 10, 4])
R

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([ 4,  7, 97, 65, 43, 43, 66,  9, 10,  4], dtype=int32)>

In [ ]:
tf.argmax(R)

<tf.Tensor: shape=(), dtype=int64, numpy=2>

In [ ]:
tf.one_hot(R, depth = 100)

<tf.Tensor: shape=(10, 100), dtype=float32, numpy=
array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.,